<a href="https://colab.research.google.com/github/rogerioag/tutorial-de-compiladores/blob/master/tppcompiler/01-compiladores-analise-lexica-tpplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Léxica

## Preparação do Ambiente

*   Instalação do [PLY](https://www.dabeaz.com/ply/ply.html)


In [ ]:
!pip install ply

     |████████████████████████████████| 51kB 2.6MB/s 


In [ ]:
!jupyter nbextension install https://rawgit.com/jfbercher/small_nbextensions/master/highlighter.zip  --user
!jupyter nbextension enable highlighter/highlighter

Downloading: https://rawgit.com/jfbercher/small_nbextensions/master/highlighter.zip -> /tmp/tmpbHxmV2/highlighter.zip
Extracting: /tmp/tmpbHxmV2/highlighter.zip -> /root/.local/share/jupyter/nbextensions
Enabling notebook extension highlighter/highlighter...
      - Validating: OK


In [ ]:
%%javascript
require("base/js/utils").load_extensions("highlighter/highlighter")

<IPython.core.display.Javascript object>

In [ ]:
from sys import argv, exit

import logging
logging.basicConfig(
     level = logging.DEBUG,
     filename = "log.txt",
     filemode = "w",
     format = "%(filename)10s:%(lineno)4d:%(message)s"
)
log = logging.getLogger()

import ply.lex as lex
from ply.lex import TOKEN

In [ ]:
tokens = [
    "ID",  # identificador
    # numerais
    "NUM_NOTACAO_CIENTIFICA",  # ponto flutuante em notaçao científica
    "NUM_PONTO_FLUTUANTE",  # ponto flutuate
    "NUM_INTEIRO",  # inteiro
    # operadores binarios
    "MAIS",  # +
    "MENOS",  # -
    "VEZES",  # *
    "DIVIDE",  # /
    "E",  # &&
    "OU",  # ||
    "DIFERENTE",  # <>
    "MENOR_IGUAL",  # <=
    "MAIOR_IGUAL",  # >=
    "MENOR",  # <
    "MAIOR",  # >
    "IGUAL",  # =
    # operadores unarios
    "NAO",  # !
    # simbolos
    "ABRE_PARENTESE",  # (
    "FECHA_PARENTESE",  # )
    "ABRE_COLCHETE",  # [
    "FECHA_COLCHETE",  # ]
    "VIRGULA",  # ,
    "DOIS_PONTOS",  # :
    "ATRIBUICAO",  # :=
    # 'COMENTARIO', # {***}
]

In [ ]:
reserved_words = {
    "se": "SE",
    "então": "ENTAO",
    "senão": "SENAO",
    "fim": "FIM",
    "repita": "REPITA",
    "flutuante": "FLUTUANTE",
    "retorna": "RETORNA",
    "até": "ATE",
    "leia": "LEIA",
    "escreva": "ESCREVA",
    "inteiro": "INTEIRO",
}

tokens = tokens + list(reserved_words.values())

In [ ]:
digito = r"([0-9])"
letra = r"([a-zA-ZáÁãÃàÀéÉíÍóÓõÕ])"
sinal = r"([\-\+]?)"

""" 
    id deve começar com uma letra
"""
id = (
    r"(" + letra + r"(" + digito + r"+|_|" + letra + r")*)"
)  # o mesmo que '((letra)(letra|_|([0-9]))*)'

# inteiro = r"(" + sinal + digito + r"+)"
# inteiro = r"(" + digito + r"+)"
inteiro = r"\d+"

flutuante = (
    # r"(" + digito + r"+\." + digito + r"+?)"
    # (([-\+]?)([0-9]+)\.([0-9]+))'
    r'\d+[eE][-+]?\d+|(\.\d+|\d+\.\d*)([eE][-+]?\d+)?'
    # r'[-+]?[0-9]+(\.([0-9]+)?)'
    #r'[+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?'
    #r"(([-\+]?)([0-9]+)\.([0-9]+))"
    )

notacao_cientifica = (
    r"(" + sinal + r"([1-9])\." + digito + r"+[eE]" + sinal + digito + r"+)"
)  # o mesmo que '(([-\+]?)([1-9])\.([0-9])+[eE]([-\+]?)([0-9]+))'

In [ ]:
 
# Expressões Regulaes para tokens simples.
# Símbolos.
t_MAIS = r'\+'
t_MENOS = r'-'
t_VEZES = r'\*'
t_DIVIDE = r'/'
t_ABRE_PARENTESE = r'\('
t_FECHA_PARENTESE = r'\)'
t_ABRE_COLCHETE = r'\['
t_FECHA_COLCHETE = r'\]'
t_VIRGULA = r','
t_ATRIBUICAO = r':='
t_DOIS_PONTOS = r':'

# Operadores Lógicos.
t_E = r'&&'
t_OU = r'\|\|'
t_NAO = r'!'

# Operadores Relacionais.
t_DIFERENTE = r'<>'
t_MENOR_IGUAL = r'<='
t_MAIOR_IGUAL = r'>='
t_MENOR = r'<'
t_MAIOR = r'>'
t_IGUAL = r'='

In [ ]:
@TOKEN(id)
def t_ID(token):
    token.type = reserved_words.get(
        token.value, "ID"
    )  # não é necessário fazer regras/regex para cada palavra reservada
    # se o token não for uma palavra reservada automaticamente é um id
    # As palavras reservadas têm precedências sobre os ids

    return token

@TOKEN(notacao_cientifica)
def t_NUM_NOTACAO_CIENTIFICA(token):
    return token

@TOKEN(flutuante)
def t_NUM_PONTO_FLUTUANTE(token):
    return token

@TOKEN(inteiro)
def t_NUM_INTEIRO(token):
    return token

In [ ]:
t_ignore = " \t"

# t_COMENTARIO = r'(\{((.|\n)*?)\})'
# para poder contar as quebras de linha dentro dos comentarios
def t_COMENTARIO(token):
    r"(\{((.|\n)*?)\})"
    token.lexer.lineno += token.value.count("\n")
    # return token

def t_newline(token):
    r"\n+"
    token.lexer.lineno += len(token.value)

def define_column(input, lexpos):
    begin_line = input.rfind("\n", 0, lexpos) + 1
    return (lexpos - begin_line) + 1

In [ ]:
def t_error(token):

    # file = token.lexer.filename
    line = token.lineno
    # column = define_column(token.lexer.backup_data, token.lexpos)
    message = "Caracter ilegal '%s'" % token.value[0]

    # print(f"[{file}]:[{line},{column}]: {message}.") 
    print(message)

    token.lexer.skip(1)

    # token.lexer.has_error = Trueb

In [ ]:
def main():
    # argv[1] = 'teste.tpp'
    aux = argv[1].split('.')
    if aux[-1] != 'tpp':
      raise IOError("Not a .tpp file!")
    data = open(argv[1])

    source_file = data.read()
    lexer.input(source_file)

    # Tokenize
    while True:
      tok = lexer.token()
      if not tok: 
        break      # No more input
      print(tok)
      # print(tok.type)
      #print(tok.value)

In [ ]:
# Build the lexer.
__file__ = "01-compiladores-analise-lexica-tpplex.ipynb"
lexer = lex.lex(optimize=True,debug=True,debuglog=log)

if __name__ == "__main__":
    main()

OSError: ignored

In [ ]:
%%writefile teste.tpp

inteiro: a[10]
flutuante: b

inteiro func1(inteiro:x, flutuante:y)
  inteiro: res
  se (x > y) então
    res := x + y
  senão
    res := x * y
  fim
  retorna(res)
fim

func2(inteiro:z, flutuante:w)
  a := z
  b := w
fim

inteiro principal()
  inteiro: x,y
  flutuante: w
  a := 10 + 2
  leia(x)
  leia(w)
  w := .6 + 1.
  func2(1, 2.5)
  b := func1(x,w)
  escreva(b)
  retorna(0)
fim


Writing teste.tpp


In [1]:
! wget https://raw.githubusercontent.com/rogerioag/tutorial-de-compiladores/master/tppcompiler/tpplex.py

--2021-03-10 19:12:51--  https://raw.githubusercontent.com/rogerioag/tutorial-de-compiladores/master/tppcompiler/tpplex.py
Carregou certificado CA "/etc/ssl/certs/ca-certificates.crt"
Resolvendo raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Conectando-se a raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 4661 (4,6K) [text/plain]
Salvando em: “tpplex.py.1”

tpplex.py.1         100%[===================>]   4,55K  --.-KB/s    em 0s      

2021-03-10 19:12:52 (37,4 MB/s) - “tpplex.py.1” salvo [4661/4661]



In [2]:
!python tpplex.py teste.tpp

INTEIRO
DOIS_PONTOS
ID
ABRE_COL
NUM_INTEIRO
FECHA_COL
FLUTUANTE
DOIS_PONTOS
ID
INTEIRO
ID
ABRE_PAR
INTEIRO
DOIS_PONTOS
ID
VIRGULA
FLUTUANTE
DOIS_PONTOS
ID
FECHA_PAR
INTEIRO
DOIS_PONTOS
ID
SE
ABRE_PAR
ID
MAIOR
ID
FECHA_PAR
ENTAO
ID
ATRIBUICAO
ID
ADICAO
ID
SENAO
ID
ATRIBUICAO
ID
MULTIPLICACAO
ID
FIM
RETORNA
ABRE_PAR
ID
FECHA_PAR
FIM
ID
ABRE_PAR
INTEIRO
DOIS_PONTOS
ID
VIRGULA
FLUTUANTE
DOIS_PONTOS
ID
FECHA_PAR
ID
ATRIBUICAO
ID
ID
ATRIBUICAO
ID
FIM
INTEIRO
ID
ABRE_PAR
FECHA_PAR
INTEIRO
DOIS_PONTOS
ID
VIRGULA
ID
FLUTUANTE
DOIS_PONTOS
ID
ID
ATRIBUICAO
NUM_INTEIRO
ADICAO
NUM_INTEIRO
LEIA
ABRE_PAR
ID
FECHA_PAR
LEIA
ABRE_PAR
ID
FECHA_PAR
ID
ATRIBUICAO
NUM_PONTO_FLUTUANTE
ADICAO
NUM_PONTO_FLUTUANTE
ID
ABRE_PAR
NUM_INTEIRO
VIRGULA
NUM_PONTO_FLUTUANTE
FECHA_PAR
ID
ATRIBUICAO
ID
ABRE_PAR
ID
VIRGULA
ID
FECHA_PAR
ESCREVA
ABRE_PAR
ID
FECHA_PAR
RETORNA
ABRE_PAR
NUM_INTEIRO
FECHA_PAR
FIM


In [ ]:
%%writefile teste-003.tpp
inteiro principal()
  a :=+1
  a := a + b
  b:= 3 + a
  c:= +3
fim

Overwriting teste-003.tpp


In [ ]:
!python tpplex.py teste-003.tpp

INTEIRO
ID
ABRE_PARENTESE
FECHA_PARENTESE
ID
ATRIBUICAO
MAIS
NUM_INTEIRO
ID
ATRIBUICAO
ID
MAIS
ID
ID
ATRIBUICAO
NUM_INTEIRO
MAIS
ID
ID
ATRIBUICAO
MAIS
NUM_INTEIRO
FIM
